# Validation

## Introduction

**What is a validation**

Validation is the backbone of reliable software. Essentially, it involves checking whether the output of a function matches our expectations. Traditionally, validation was deterministic and rule-based, focusing on specific criteria (for example, 'output must not be larger than a certain value'). However, with the advent of Large Language Models (LLMs), we've seen the integration of probabilistic validation into our processes. This type of validation, often labeled as 'guardrail', might appear novel, but it's essentially another form of validation that leverages LLMs instead of strict rules to flag responses.

In our approach at instructor, we treat all forms of validation equally. Whether it's rule-based, probabilistic, or a combination of both, everything is managed within a singular, cohesive framework. We achieve this through extensive use of Pydantic's powerful [validators](https://docs.pydantic.dev/latest/concepts/validators/#field-validators) feature.

Validators will enable us to control outputs by defining a function like so:

In [1]:
def validation_function(value):
    if condition(value):
        raise ValueError("Value is not valid")
    return mutation(value)

The validation process in this framework unfolds in three key steps:

* Condition Verification: The first step involves the validator checking whether a value meets a set condition. This is where the core validation logic is applied.

* Error Handling with Optional Retry: If the value doesn't meet the condition, the system raises an error. There's an option to retry, offering a chance for correcting and reevaluating the value.

* Value Processing: When the value meets the condition, the validator returns either the original or a modified version of the value. This ensures the output is valid and meets specific requirements or preferences.

**Validation Applications**

Validators are essential in tackling the unpredictabile nature of LLMs.

Straightforward examples include:

* Flagging outputs containing blacklisted words.
* Identifying outputs with tones like racism or violence.

For more complex tasks:

* Ensuring citations directly come from provided content.
* Checking that the model's responses align with given context.
* Validating the syntax of SQL queries before execution.

## Setup and Dependencies

Using the [instructor](https://github.com/jxnl/instructor) library, we streamline the integration of these validators. `instructor` manages the parsing and validation of outputs and automates retries for compliant responses. This simplifies the process for developers to implement new validation logic, minimizing extra overhead.

To use instructor in our api calls, we just need to patch the openai client:

In [3]:
import instructor 
from openai import OpenAI

client = instructor.patch(OpenAI())

## Software 2.0: Rule-based validators

Deterministic validation, characterized by its rule-based logic, ensures consistent outcomes for the same input. Let's explore how we can apply this concept through some examples.

### Flagging bad keywords

To begin with, we aim to prevent engagement in topics involving explicit violence.

We will define a blacklist of violent words that cannot be mentioned in any messages:

In [4]:
blacklist = {
    "rob",
    "steal",
    "hurt",
    "kill",
    "attack",
}

To validate if the message contains a blacklisted word we will use a [field_validator](https://jxnl.github.io/instructor/blog/2023/10/23/good-llm-validation-is-just-good-validation/#using-field_validator-decorator) over the 'message' field:

In [5]:
from pydantic import BaseModel, ValidationError, field_validator
from pydantic.fields import Field

class UserMessage(BaseModel):
    message: str

    @field_validator('message')
    def message_cannot_have_blacklisted_words(cls, v: str) -> str:
        for word in v.split(): 
            if word.lower() in blacklist:
                raise ValueError(f"`{word}` was found in the message `{v}`")
        return v

try:
    UserMessage(message="I will hurt him")
except ValidationError as e:
    print(e)

1 validation error for UserMessage
message
  Value error, `hurt` was found in the message `I will hurt him` [type=value_error, input_value='I will hurt him', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error


### Flagging using OpenAI Moderation

To enhance our validation measures, we'll extend the scope to flag any answer that contains hateful content, harassment, or similar issues. OpenAI offers a moderation endpoint that addresses these concerns, and it's freely available when using OpenAI models.

With the `instructor` library, this is just one function edit away:

In [6]:
class UserMessage(BaseModel):
    message: str

    @field_validator('message')
    def message_must_comply_with_openai_mod(cls, v: str) -> str:
        response = client.moderations.create(input=v)
        out = response.results[0]
        cats = dict(out.categories)
        if out.flagged:
            raise ValueError(f"`{v}` was flagged for {[i for i in cats if cats[i]]}")
        
        return v 

Now we have a more comprehensive flagging for violence and we can outsource the moderation of our messages.

In [7]:
try:
    UserMessage(message="I want to make them suffer the consequences")
except ValidationError as e:
    print(e)

1 validation error for UserMessage
message
  Value error, `I want to make them suffer the consequences` was flagged for ['harassment', 'harassment_threatening', 'violence', 'harassment/threatening'] [type=value_error, input_value='I want to make them suffer the consequences', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error


And as an extra, we get flagging for other topics like religion, race etc.

In [8]:
try:
    UserMessage(message="I will mock their religion")
except ValidationError as e:
    print(e)

1 validation error for UserMessage
message
  Value error, `I will mock their religion` was flagged for ['harassment'] [type=value_error, input_value='I will mock their religion', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error


### Filtering very long messages

In addition to content-based flags, we can also set criteria based on other aspects of the input text. For instance, to maintain user engagement, we might want to prevent the assistant from returning excessively long texts. 

We can implement this using `instructor` to set a maximum word or character limit on the assistant's responses:

In [10]:
class AssistantMessage(BaseModel):
    message: str

    @field_validator('message')
    def message_must_be_short(cls, v: str) -> str:
        if len(v.split())>=100:
            raise ValueError(f"Text was flagged for being longer than 100 words.")
        
        return v     

In [11]:
try:
    AssistantMessage(message="""
    Certainly! Lorem ipsum is a placeholder text commonly used in the printing and typesetting industry. Here's a sample of Lorem ipsum text:

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam euismod velit vel tellus tempor, non viverra eros iaculis. Sed vel nisl nec mauris bibendum tincidunt. Vestibulum sed libero euismod, eleifend tellus id, laoreet elit. Donec auctor arcu ac mi feugiat, vel lobortis justo efficitur. Fusce vel odio vitae justo varius dignissim. Integer sollicitudin mi a justo bibendum ultrices. Quisque id nisl a lectus venenatis luctus.

Please note that Lorem ipsum text is a nonsensical Latin-like text used as a placeholder for content, and it has no specific meaning. It's often used in design and publishing to demonstrate the visual aspects of a document without focusing on the actual content."""
                    )
except ValidationError as e:
    print(e)

1 validation error for AssistantMessage
message
  Value error, Text was flagged for being longer than 100 words. [type=value_error, input_value="\n    Certainly! Lorem i... on the actual content.", input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error


### Avoiding hallucination

When incorporating external knowledge bases, it's crucial to ensure that the agent uses the provided context accurately and doesn't fabricate responses. Validators can be effectively used for this purpose. 

We can illustrate this with an example where we validate that a provided citation is actually included in the referenced text chunk:

In [13]:
from pydantic import ValidationInfo

class AnswerWithCitation(BaseModel):
    answer: str
    citation: str

    @field_validator('citation')
    @classmethod
    def citation_exists(cls, v: str, info: ValidationInfo): 
        context = info.context
        if context:
            context = context.get('text_chunk')
            if v not in context:
                raise ValueError(f"Citation `{v}` not found in text chunks")
        return v

When the model responds with information not present in the provided context, the validation process comes into play:

In [14]:
try:
    AnswerWithCitation.model_validate(
        {"answer": "Blueberries are packed with protein", "citation": "Blueberries contain high levels of protein"},
        context={"text_chunk": "Blueberries are very rich in antioxidants"}, 
    )
except ValidationError as e:
    print(e)

1 validation error for AnswerWithCitation
citation
  Value error, Citation `Blueberries contain high levels of protein` not found in text chunks [type=value_error, input_value='Blueberries contain high levels of protein', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error


## Software 3.0: Probabilistic validators

For scenarios requiring more nuanced validation than what rule-based methods offer, we turn to probabilistic validation. This approach utilizes LLMs as a core component of the validation workflow, enabling a more sophisticated assessment of outputs.

The `instructor` library facilitates this with its `llm_validator` utility. By specifying the desired directive, we can employ LLMs for complex validation tasks. Let's explore some intriguing use cases that are made possible through LLMs.

### Keeping an agent on topic

In the case of creating an agent focused on health improvement, providing answers and daily practice suggestions, it's vital to ensure that the agent strictly adheres to health-related topics. This is important because the knowledge base is limited to health topics, and venturing beyond this scope could lead to  fabricated ('hallucinated') responses.

To achieve this focus, we'll employ a similar process to the one previously discussed, but with an important addition: integrating an LLM into our validator. 

This LLM will be tasked with determining whether the agent's responses are exclusively related to health topics. For this, we will use the `llm_validator` from `instructor` like so:

In [18]:
from typing import Annotated
from pydantic.functional_validators import AfterValidator
from instructor import llm_validator

class AssistantMessage(BaseModel):
    message: Annotated[str, AfterValidator(llm_validator("don't talk about any other topic except health best practices and topics"))]

try:
    AssistantMessage(message="I would suggest you to visit Sicily as they say it is very nice in winter.")
except ValidationError as e:
    print(e)

1 validation error for AssistantMessage
message
  Assertion failed, The statement is not related to health best practices or topics. [type=assertion_error, input_value='I would suggest you to v...is very nice in winter.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/assertion_error


Great! With these validations, the model will reliably stick to its knowledge base, ensuring accurate and topic-specific responses.

### Validating agent thinking with CoT

Using probabilistic validation, we can also assess the agent's reasoning process to ensure it's logical before providing a response. With [chain of thought](https://learnprompting.org/docs/intermediate/chain_of_thought) prompting, the model is expected to think in steps and arrive at an answer following its logical progression. If there are errors in this logic, the final response may be incorrect.

Here we will use Pydantic's [model_validator](https://docs.pydantic.dev/latest/concepts/validators/#model-validators) which allows us to apply validation over all the properties of the `AIResponse` at once.

To achieve this, we'll create a `Validation` class that specifies the desired output format from our LLM call, similar to how `llm_validator` functioned in the earlier example.


This `Validation` class will be used to determine if the chain of thought in the model's response is valid. If it's not valid, the class will also provide an explanation as to why:

In [39]:
from typing import Optional

class Validation(BaseModel):
    is_valid: bool = Field(..., description="Whether the value is valid based on the rules")
    error_message: Optional[str] = Field(..., description="The error message if the value is not valid, to be used for re-asking the model")

The function we will call will integrate an LLM and will ask it to determine whether the answer the model provided follows from the chain of thought: 

In [40]:
def validate_chain_of_thought(values):
    chain_of_thought = values["chain_of_thought"]
    answer = values["answer"]
    resp = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a validator. Determine if the value follows from the statement. If it is not, explain why.",
            },
            {
                "role": "user",
                "content": f"Verify that `{answer}` follows the chain of thought: {chain_of_thought}",
            },
        ],
        # this comes from client = instructor.patch(OpenAI())
        response_model=Validation,
    )
    print(resp)
    if not resp.is_valid:
        raise ValueError(resp.error_message)
    return values

The use of the 'before' argument in this context is significant. It means that the validator will receive the complete dictionary of inputs in their raw form, before any parsing by Pydantic.

In [41]:
from typing import Any
from pydantic import model_validator

class AIResponse(BaseModel):
    chain_of_thought: str
    answer: str

    @model_validator(mode='before')
    @classmethod
    def chain_of_thought_makes_sense(cls, data: Any) -> Any:
        # here we assume data is the dict representation of the model
        # since we use 'before' mode.
        return validate_chain_of_thought(data)

In [42]:
try:
    resp = AIResponse(
        chain_of_thought="The user suffers from diabetes.", answer="The user has a broken leg."
)
except ValidationError as e:
    print(e)

is_valid=False error_message='The user has a broken leg which is not related to diabetes.'
1 validation error for AIResponse
  Value error, The user has a broken leg which is not related to diabetes. [type=value_error, input_value={'chain_of_thought': 'The...user has a broken leg.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error


## Using validation

Integrating these validation examples with the OpenAI API is streamlined using `instructor`. After patching the OpenAI client with `instructor`, you simply need to specify a `response_model` for your requests. This setup ensures that all the validation processes occur automatically.

Additionally, you can set a maximum number of retries. When calling the OpenAI client, the system can re-attempt to generate a correct answer. It does this by resending the original query along with feedback on why the previous response was rejected, guiding the LLM towards a more accurate answer in subsequent attempts.

In [32]:
class HealthAnswer(BaseModel):
    answer: Annotated[str, AfterValidator(llm_validator("don't talk about any other topic except health best practices and topics"))]

In [33]:
try:
    model = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Which is the best headphone brand for producing music?"},
    ],
    response_model=HealthAnswer,
    max_retries=2,
)
except ValidationError as e:
    print(e)

1 validation error for HealthAnswer
answer
  Assertion failed, The statement is not related to health best practices or topics. [type=assertion_error, input_value="While there isn't a sing...aking a final decision.", input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/assertion_error


# Conclusion

This guide showed how to use deterministic and probabilistic validation techniques with Large Language Models. We covered using instructor to set up validation processes for filtering content, maintaining context relevance, and checking model reasoning. These methods improve the performance of LLMs across various tasks.

For those looking to delve deeper here's a to-do list to explore:

1. **SQL Syntax Checker**: Create a validator to check SQL query syntax before execution.
2. **Context-Based Response Validation**: Design a method to flag responses based on the model's own knowledge rather than the provided context.
3. **PII Detection**: Implement a mechanism to identify and handle Personally Identifiable Information in responses, focusing on maintaining user privacy.
4. **Targeted Rule-Based Filtering**: Develop filters to remove certain content types, like responses mentioning named entities.
    
Completing these tasks will help users gain practical skills in enhancing LLMs through advanced validation methods.